In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, cheby1, cheby2
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from tkinter import filedialog
import scipy.io.wavfile as wavfile

# Parámetros iniciales
N = 4096  # Número de muestras
fs = 48000  # Frecuencia de muestreo por defecto

# Variables para determinar si una señal es ideal o proviene de un archivo WAV
es_ideal_onda1 = True
es_ideal_onda2 = True

# Variables globales para las señales
onda1_original = None
onda2_original = None
fs1 = fs  # Frecuencia de muestreo de onda1
fs2 = fs  # Frecuencia de muestreo de onda2

# Función para crear las señales onda1 y onda2 con ganancia y polaridad
def crear_ondas(N, ganancia_onda1_db, ganancia_onda2_db, polaridad_onda1, polaridad_onda2):
    # Inicializar las señales
    onda1 = np.zeros(N)  # Inicializar la señal Onda 1
    onda2 = np.zeros(N)  # Inicializar la señal Onda 2

    # Crear señales impulsionales con ganancia y polaridad aplicadas
    onda1[0] = 10 ** (ganancia_onda1_db / 20) * np.cos(polaridad_onda1)
    onda2[0] = 10 ** (ganancia_onda2_db / 20) * np.cos(polaridad_onda2)

    return onda1, onda2

# Función para cargar archivo WAV y sobrescribir la señal ideal
def cargar_wav(canal):
    global es_ideal_onda1, es_ideal_onda2, onda1_original, onda2_original
    archivo_wav = filedialog.askopenfilename(filetypes=[("Archivos WAV", "*.wav")])
    
    if archivo_wav:
        try:
            fs, data = wavfile.read(archivo_wav)

            if len(data.shape) == 2:  # Señal estéreo
                if canal == 1:
                    data_canal = data[:, 0]
                else:
                    data_canal = data[:, 1]
            else:  # Señal mono
                data_canal = data
            
            if len(data_canal) > N:
                data_canal = data_canal[:N]
            elif len(data_canal) < N:
                data_canal = np.pad(data_canal, (0, N - len(data_canal)), 'constant')

            if canal == 1:
                global onda1, fs1
                fs1 = fs
                onda1_original = data_canal  # Guardar la señal original para procesarla luego
                onda1 = onda1_original.copy()  # Usar una copia de la señal original
                es_ideal_onda1 = False  # Señal cargada desde WAV
            else:
                global onda2, fs2
                fs2 = fs
                onda2_original = data_canal  # Guardar la señal original para procesarla luego
                onda2 = onda2_original.copy()  # Usar una copia de la señal original
                es_ideal_onda2 = False  # Señal cargada desde WAV

            actualizar_señales()
        except Exception as e:
            error_label.config(text=f"Error al cargar el archivo WAV: {str(e)}")

def seleccionar_filtro(tipo_filtro, order, cutoff, fs, tipo, ripple=None, bandwidth=None):
    if tipo_filtro == "Butterworth":
        return butter(order, cutoff / (0.5 * fs), btype=tipo)
    elif tipo_filtro == "Chebyshev 1":
        if ripple is None:
            ripple = 1  # Valor predeterminado de ripple si no se proporciona
        return cheby1(order, ripple, cutoff / (0.5 * fs), btype=tipo)
    elif tipo_filtro == "Chebyshev 2":
        if ripple is None:
            ripple = 40  # Valor predeterminado de ripple (atenuación) para Chebyshev 2
        return cheby2(order, ripple, cutoff / (0.5 * fs), btype=tipo)
    elif tipo_filtro == "Linkwitz-Riley":
        return butter(order // 2, cutoff / (0.5 * fs), btype=tipo)
    elif tipo_filtro == "All Pass":
        if order == 1:
            return all_pass_1st_order(cutoff, fs)
        elif order == 2:
            if bandwidth is None:
                bandwidth = 0.707  # Usar bandwidth (Q) como factor de calidad para All Pass de segundo orden
            return all_pass_2nd_order(cutoff, fs, bandwidth)
        else:
            raise ValueError("El filtro All Pass solo admite orden 1 o 2.")
    else:
        raise ValueError("Tipo de filtro no válido")

def all_pass_1st_order(cutoff, fs):
    omega = 2 * np.pi * cutoff / fs  # Frecuencia angular normalizada
    alpha = (1 - np.sin(omega)) / np.cos(omega)
    # Coeficientes del filtro All Pass de primer orden
    b = [alpha, -1]
    a = [1, -alpha]
    return b, a

def all_pass_2nd_order(cutoff, fs, Q=0.707):
    omega = 2 * np.pi * cutoff / fs
    alpha = np.sin(omega) / (2 * Q)

    a0 = 1 + alpha
    a1 = -2 * np.cos(omega)
    a2 = 1 - alpha

    b0 = a2
    b1 = a1
    b2 = a0

    b = [b0 / a0, b1 / a0, b2 / a0]
    a = [1, a1 / a0, a2 / a0]

    return b, a
    
def obtener_ripple(entry):
    try:
        valor = float(entry.get())
        if valor > 0:
            return valor
        else:
            raise ValueError
    except ValueError:
        error_label.config(text="Valor de ripple no válido. Se usará el valor predeterminado de 1 dB.")
        return 1

def obtener_bandwidth(entry):
    try:
        valor = float(entry.get())
        if valor > 0:
            return valor
        else:
            raise ValueError
    except ValueError:
        error_label.config(text="Valor de ancho de banda no válido. Se usará el valor predeterminado de 0.707.")
        return 0.707

def obtener_orden_filtro(entry):
    try:
        valor = int(entry.get())
        if valor >= 1:  # Solo aceptamos valores enteros positivos
            return valor
        else:
            raise ValueError
    except ValueError:
        error_label.config(text="Orden no válido. Se usará el valor predeterminado de 1.")
        return 1  # Valor predeterminado si no es válido

def aplicar_delay(signal, fs, delay_ms):
    delay_samples = int(delay_ms * fs / 1000)
    if delay_samples == 0:
        return signal
    delayed_signal = np.zeros_like(signal)
    if delay_samples < len(signal):
        delayed_signal[delay_samples:] = signal[:-delay_samples]
    return delayed_signal

def obtener_delay_desde_texto(texto):
    try:
        valor = float(texto.get())
        return valor if valor >= 0 else 0
    except ValueError:
        return 0.0

def obtener_frecuencia_corte(entry):
    try:
        valor = float(entry.get())
        if 20 <= valor <= 20000:
            return valor
        else:
            raise ValueError
    except ValueError:
        if entry.get().strip() == "":  # Si el campo está vacío
            return 20  # Valor predeterminado
        error_label.config(text="Frecuencia no válida. Se usará el valor predeterminado de 20 Hz.")
        return 20

# Función para actualizar las señales y aplicar transformaciones
def actualizar_señales(*args):
    global onda1, onda2, es_ideal_onda1, es_ideal_onda2

    error_label.config(text="")  # Limpiar mensaje de error

    # Usar las frecuencias de muestreo correspondientes (si se ha cargado WAV o usar por defecto)
    fs_onda1 = fs1 if not es_ideal_onda1 else fs
    fs_onda2 = fs2 if not es_ideal_onda2 else fs

    # Obtener valores de la interfaz
    ganancia_onda1_db = ganancia1_scale.get()
    ganancia_onda2_db = ganancia2_scale.get()
    polaridad_onda1 = 0 if polaridad1_var.get() == "0" else np.pi
    polaridad_onda2 = 0 if polaridad2_var.get() == "0" else np.pi

    filtro1_tipo = filtro1_var.get()
    filtro2_tipo = filtro2_var.get()
    tipo1_filtro = tipo1_var.get()
    tipo2_filtro = tipo2_var.get()

    orden1 = obtener_orden_filtro(orden1_entry)
    orden2 = obtener_orden_filtro(orden2_entry)

    cutoff1 = obtener_frecuencia_corte(cutoff1_entry)
    cutoff2 = obtener_frecuencia_corte(cutoff2_entry)

    delay1_ms = obtener_delay_desde_texto(delay1_text)
    delay2_ms = obtener_delay_desde_texto(delay2_text)

    ripple1 = obtener_ripple(riple_entry) if "Chebyshev" in filtro1_tipo else None
    ripple2 = obtener_ripple(riple_entry) if "Chebyshev" in filtro2_tipo else None

    bandwidth1 = obtener_bandwidth(bandwidth_entry) if filtro1_var.get() == "All Pass" else None
    bandwidth2 = obtener_bandwidth(bandwidth_entry) if filtro2_var.get() == "All Pass" else None

    # Restaurar la señal original antes de aplicar el filtro
    if es_ideal_onda1:
        onda1, _ = crear_ondas(N, ganancia_onda1_db, 0, polaridad_onda1, 0)
    else:
        onda1 = onda1_original.copy()  # Restaurar la señal original cargada desde el archivo

    if es_ideal_onda2:
        _, onda2 = crear_ondas(N, 0, ganancia_onda2_db, 0, polaridad_onda2)
    else:
        onda2 = onda2_original.copy()  # Restaurar la señal original cargada desde el archivo

    # Aplicar ganancia y polaridad
    # Crear señales impulsionales con ganancia y polaridad aplicadas
    onda1 = np.zeros(N)
    onda2 = np.zeros(N)
    onda1[0] = 10 ** (ganancia_onda1_db / 20) * np.cos(polaridad_onda1)
    onda2[0] = 10 ** (ganancia_onda2_db / 20) * np.cos(polaridad_onda2)

    # Aplicar filtro a Onda 1
    if filtro1_tipo != "none":
        b, a = seleccionar_filtro(filtro1_tipo, orden1, cutoff1, fs_onda1, tipo1_filtro, ripple=ripple1, bandwidth=bandwidth1)
        onda1 = lfilter(b, a, onda1)

    # Aplicar filtro a Onda 2
    if filtro2_tipo != "none":
        b, a = seleccionar_filtro(filtro2_tipo, orden2, cutoff2, fs_onda2, tipo2_filtro, ripple=ripple2, bandwidth=bandwidth2)
        onda2 = lfilter(b, a, onda2)

    # Aplicar el delay a ambas señales
    onda1 = aplicar_delay(onda1, fs_onda1, delay1_ms)
    onda2 = aplicar_delay(onda2, fs_onda2, delay2_ms)

    # FFT de las ondas para obtener magnitud y fase
    fft_onda1 = np.fft.fft(onda1)
    fft_onda2 = np.fft.fft(onda2)

    # Calcular la suma y el promedio de las señales
    suma_ondas = onda1 + onda2
    promedio_ondas = (onda1 + onda2) / 2

    # FFT de la sumatoria y promedio
    fft_suma = np.fft.fft(suma_ondas)
    fft_promedio = np.fft.fft(promedio_ondas)

    # Magnitud y fase
    magnitud_onda1 = np.abs(fft_onda1)
    magnitud_onda2 = np.abs(fft_onda2)
    magnitud_suma = np.abs(fft_suma)
    magnitud_promedio = np.abs(fft_promedio)

    fase_onda1 = np.angle(fft_onda1) * 180 / np.pi
    fase_onda2 = np.angle(fft_onda2) * 180 / np.pi
    fase_suma = np.angle(fft_suma) * 180 / np.pi
    fase_promedio = np.angle(fft_promedio) * 180 / np.pi

    frequencies = np.fft.fftfreq(N, 1/fs_onda1)

    # Filtrar las frecuencias para mostrar solo de 20 Hz a 20 kHz
    min_freq = 20
    max_freq = 20000
    idx = np.where((frequencies >= min_freq) & (frequencies <= max_freq))

    # Limpiar los ejes antes de volver a dibujar
    ax1.clear()
    ax2.clear()

    epsilon = 1e-10

    # Gráfica combinada de Magnitud en escala logarítmica
    ax1.semilogx(frequencies[idx], 20 * np.log10(magnitud_onda1[idx] + epsilon), label='Onda 1')
    ax1.semilogx(frequencies[idx], 20 * np.log10(magnitud_onda2[idx] + epsilon), label='Onda 2')
    ax1.semilogx(frequencies[idx], 20 * np.log10(magnitud_suma[idx] + epsilon), label='Suma')
    ax1.semilogx(frequencies[idx], 20 * np.log10(magnitud_promedio[idx] + epsilon), label='Promedio')
    ax1.axhline(-3, color='black', linestyle='--', linewidth=0.7)  # Línea de -3 dB
    ax1.set_xlabel('Frecuencia (Hz)')
    ax1.set_xlim([20, 20000])
    ax1.set_ylabel('Magnitud (dB)')
    ax1.set_title("Magnitud de las Ondas [dB]")
    ax1.legend()
    ax1.grid(True)

    xtick_labels = [31.5, 63, 125, 250, 500, 1000, 4000, 8000, 16000]
    xtick_labels_str = ['31.5', '63', '125', '250', '500', '1k', '4k', '8k', '16k']

    ax1.set_xticks(xtick_labels)
    ax1.set_xticklabels(xtick_labels_str)

    # Gráfica combinada de Fase
    ax2.semilogx(frequencies[idx], fase_onda1[idx], label='Onda 1')
    ax2.semilogx(frequencies[idx], fase_onda2[idx], label='Onda 2')
    ax2.semilogx(frequencies[idx], fase_suma[idx], label='Suma')
    ax2.semilogx(frequencies[idx], fase_promedio[idx], label='Promedio')
    ax2.set_xlabel('Frecuencia (Hz)')
    ax2.set_yticks([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    ax2.set_xlim([20, 20000])
    ax2.set_ylim([-185, 185])
    ax2.set_ylabel('Fase (°)')
    ax2.set_title("Fase de las Ondas (Grados)")
    ax2.legend()
    ax2.grid(True)
    ax2.set_xticks(xtick_labels)
    ax2.set_xticklabels(xtick_labels_str)

    # Ajustar el layout para evitar la sobreposición
    fig.tight_layout()

    # Redibujar los gráficos
    canvas.draw()


# Configuración de la interfaz gráfica con Tkinter
root = tk.Tk()
root.title("Control de Señales")

# Establecer el color del texto de los Combobox a negro
style = ttk.Style()
style.configure("TCombobox", foreground="black")

root.geometry("1440x900")

# Crear figura y ejes para la gráfica
fig = Figure(figsize=(9, 7))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

ax1.set_xlabel('Frecuencia (Hz)')
ax1.set_ylabel('Magnitud (dB)')
ax1.grid(True)

ax2.set_xlabel('Frecuencia (Hz)')
ax2.set_ylabel('Fase (°)')
ax2.grid(True)

fig.tight_layout()

canvas = FigureCanvasTkAgg(fig, master=root)
canvas.draw()
canvas.get_tk_widget().grid(row=0, column=2, rowspan=14, padx=10, pady=10, sticky="nsew")

tk.Label(root, text="Ganancia Onda 1 (dB)").grid(row=0, column=0, sticky="w", padx=10)
ganancia1_scale = tk.Scale(root, from_=-30, to=30, resolution=1, orient="horizontal")
ganancia1_scale.set(0)
ganancia1_scale.grid(row=0, column=1, padx=10, pady=5)
ganancia1_scale.bind("<ButtonRelease-1>", actualizar_señales)

tk.Label(root, text="Ganancia Onda 2 (dB)").grid(row=1, column=0, sticky="w", padx=10)
ganancia2_scale = tk.Scale(root, from_=-30, to=30, resolution=1, orient="horizontal")
ganancia2_scale.set(0)
ganancia2_scale.grid(row=1, column=1, padx=10, pady=5)
ganancia2_scale.bind("<ButtonRelease-1>", actualizar_señales)

tk.Label(root, text="Polaridad Onda 1").grid(row=2, column=0, sticky="w", padx=10)
polaridad1_var = tk.StringVar(value="0")
ttk.Combobox(root, textvariable=polaridad1_var, values=["0", "180º"]).grid(row=2, column=1, padx=10)
polaridad1_var.trace("w", actualizar_señales)

tk.Label(root, text="Polaridad Onda 2").grid(row=3, column=0, sticky="w", padx=10)
polaridad2_var = tk.StringVar(value="0")
ttk.Combobox(root, textvariable=polaridad2_var, values=["0", "180º"]).grid(row=3, column=1, padx=10)
polaridad2_var.trace("w", actualizar_señales)

# Filtro y orden para Onda 1
tk.Label(root, text="Filtro Onda 1").grid(row=4, column=0, sticky="w", padx=10)
filtro1_var = tk.StringVar(value="none")
ttk.Combobox(root, textvariable=filtro1_var, values=["none", "Butterworth", "Chebyshev 1", "Chebyshev 2", "Linkwitz-Riley", "All Pass"], style="TCombobox").grid(row=4, column=1, padx=10)
filtro1_var.trace("w", actualizar_señales)

tk.Label(root, text="Orden Filtro Onda 1").grid(row=5, column=0, sticky="w", padx=10)
orden1_entry = tk.Entry(root)
orden1_entry.grid(row=5, column=1, padx=10)
orden1_entry.insert(0, "1")
orden1_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Frecuencia de Corte Onda 1 (Hz)").grid(row=6, column=0, sticky="w", padx=10)
cutoff1_entry = tk.Entry(root)
cutoff1_entry.grid(row=6, column=1, padx=10)
cutoff1_entry.insert(0, "1000")
cutoff1_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Tipo de Paso Onda 1").grid(row=7, column=0, sticky="w", padx=10)
tipo1_var = tk.StringVar(value="none")
ttk.Combobox(root, textvariable=tipo1_var, values=["none", "lowpass", "highpass"], style="TCombobox").grid(row=7, column=1, padx=10)
tipo1_var.trace("w", actualizar_señales)

# Filtro y orden para Onda 2
tk.Label(root, text="Filtro Onda 2").grid(row=8, column=0, sticky="w", padx=10)
filtro2_var = tk.StringVar(value="none")
ttk.Combobox(root, textvariable=filtro2_var, values=["none", "Butterworth", "Chebyshev 1", "Chebyshev 2", "Linkwitz-Riley", "All Pass"], style="TCombobox").grid(row=8, column=1, padx=10)
filtro2_var.trace("w", actualizar_señales)

tk.Label(root, text="Orden Filtro Onda 2").grid(row=9, column=0, sticky="w", padx=10)
orden2_entry = tk.Entry(root)
orden2_entry.grid(row=9, column=1, padx=10)
orden2_entry.insert(0, "1")
orden2_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Frecuencia de Corte Onda 2 (Hz)").grid(row=10, column=0, sticky="w", padx=10)
cutoff2_entry = tk.Entry(root)
cutoff2_entry.grid(row=10, column=1, padx=10)
cutoff2_entry.insert(0, "1000")
cutoff2_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Tipo de Paso Onda 2").grid(row=11, column=0, sticky="w", padx=10)
tipo2_var = tk.StringVar(value="none")
ttk.Combobox(root, textvariable=tipo2_var, values=["none", "lowpass", "highpass"], style="TCombobox").grid(row=11, column=1, padx=10)
tipo2_var.trace("w", actualizar_señales)

tk.Label(root, text="Delay Onda 1 (ms)").grid(row=12, column=0, sticky="w", padx=10)
delay1_text = tk.Entry(root)
delay1_text.grid(row=12, column=1, padx=10)
delay1_text.insert(0, "0")
delay1_text.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Delay Onda 2 (ms)").grid(row=13, column=0, sticky="w", padx=10)
delay2_text = tk.Entry(root)
delay2_text.grid(row=13, column=1, padx=10)
delay2_text.insert(0, "0")
delay2_text.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Ancho de Banda").grid(row=14, column=0, sticky="w", padx=10)
bandwidth_entry = tk.Entry(root)
bandwidth_entry.grid(row=14, column=1, padx=10)
bandwidth_entry.insert(0, "1")
bandwidth_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Ripple").grid(row=15, column=0, sticky="w", padx=10)
riple_entry = tk.Entry(root)
riple_entry.grid(row=15, column=1, padx=10)
riple_entry.insert(0, "1")
riple_entry.bind("<KeyRelease>", actualizar_señales)

tk.Label(root, text="Cargar WAV Onda 1").grid(row=16, column=0, sticky="w", padx=10)
boton_wav_onda1 = tk.Button(root, text="Cargar WAV", command=lambda: cargar_wav(1))
boton_wav_onda1.grid(row=16, column=1, padx=10, pady=5)

tk.Label(root, text="Cargar WAV Onda 2").grid(row=17, column=0, sticky="w", padx=10)
boton_wav_onda2 = tk.Button(root, text="Cargar WAV", command=lambda: cargar_wav(2))
boton_wav_onda2.grid(row=17, column=1, padx=10, pady=5)

error_label = tk.Label(root, text="", fg="red")
error_label.grid(row=17, column=1)

actualizar_señales()

root.mainloop()
